
# 09: Feature Engineering Basics (ML Bridge)

このノートでは、OHLCVから**機械学習用の特徴量**を作る基本を学びます。

学ぶこと:
- リターン・ローリング平均/標準偏差・EMA差・ATR・レンジ・出来高変化
- 正規化特徴量 `ema_diff / atr`
- 相関ヒートマップと分布可視化
- 「どの特徴が何を表すか」の読み解き



## Concept と式

代表的な特徴量の式:

- 日次リターン: $r_t = rac{C_t}{C_{t-1}} - 1$
- EMA差分(トレンド): $	ext{ema\_diff}_t = EMA_{12}(C_t) - EMA_{26}(C_t)$
- True Range: $TR_t = \max(H_t-L_t, |H_t-C_{t-1}|, |L_t-C_{t-1}|)$
- ATR(14): $ATR_t = rac{1}{14}\sum_{i=0}^{13}TR_{t-i}$
- 正規化: $	ext{ema\_diff\_over\_atr}_t = rac{	ext{ema\_diff}_t}{ATR_t}$

> ポイント: 正規化を行うと、銘柄や期間でボラティリティ水準が異なる場合でも比較しやすくなります。


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from quantlab.ml_bridge import build_feature_frame

np.random.seed(42)
dates = pd.date_range("2023-01-01", periods=320, freq="B")

# 教材用の疑似OHLCVデータ（ネットワーク不要で再現可能）
base = 100 + np.cumsum(np.random.normal(0.03, 1.0, len(dates)))
close = pd.Series(base, index=dates)
open_ = close.shift(1).fillna(close.iloc[0]) + np.random.normal(0, 0.4, len(dates))
spread = np.abs(np.random.normal(0.8, 0.3, len(dates)))
high = np.maximum(open_, close) + spread
low = np.minimum(open_, close) - spread
volume = (1_000_000 + np.random.normal(0, 130_000, len(dates))).clip(100_000)

ohlcv = pd.DataFrame({
    "Open": open_,
    "High": high,
    "Low": low,
    "Close": close,
    "Volume": volume,
}, index=dates)

features = build_feature_frame(ohlcv).dropna()
print("Feature dataframe shape:", features.shape)
features.head()


## 数値例（サンプル行）と解説

In [ ]:

    sample = features[[
        "ret_1d", "roll_std_20", "ema_diff", "atr_14", "ema_diff_over_atr", "volume_change_1d"
    ]].head(8).round(4)

    print("
--- sample rows ---")
    print(sample)

    print("
解説:")
    print("- ret_1d: 1日の値動き率")
    print("- roll_std_20: 20日ボラティリティの近似")
    print("- ema_diff: 短期EMAと長期EMAの差（トレンド方向）")
    print("- atr_14: 価格レンジ由来の変動幅")
    print("- ema_diff_over_atr: トレンド強さをボラで割った規格化指標")


## Correlation 可視化

In [ ]:

corr = features.corr(numeric_only=True)

fig, ax = plt.subplots(figsize=(9, 7))
im = ax.imshow(corr.values, cmap="coolwarm", vmin=-1, vmax=1)
ax.set_xticks(range(len(corr.columns)))
ax.set_yticks(range(len(corr.columns)))
ax.set_xticklabels(corr.columns, rotation=90)
ax.set_yticklabels(corr.columns)
ax.set_title("Feature Correlation Heatmap")
fig.colorbar(im, ax=ax, shrink=0.8)
plt.tight_layout()
plt.show()


## Distribution 可視化

In [ ]:

cols = ["ret_1d", "roll_std_20", "ema_diff_over_atr", "range_close", "volume_change_1d"]
fig, axes = plt.subplots(2, 3, figsize=(12, 6))
axes = axes.ravel()

for i, col in enumerate(cols):
    axes[i].hist(features[col].dropna(), bins=30, alpha=0.8)
    axes[i].set_title(col)

axes[-1].axis("off")
plt.tight_layout()
plt.show()



## What to observe

- `ema_diff` と `ema_diff_over_atr` は似た動きをしつつ、後者はボラティリティ調整で外れ値が抑えられます。
- `ret_1d` は平均0付近に集中し、`volume_change_1d` は裾が厚くなりやすいです。
- 相関が高すぎる特徴量を多重に使うと、モデルが冗長になりやすいので注意しましょう。

関連ノート: `notebooks/10_walk_forward_validation.ipynb`
